In [ ]:
pip install pulp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 28.8 MB/s eta 0:00:00


**Nomenclature**:

Terminal 1 and 2 .....
Vessel 1 and 2.....
Product 1 and 2

*   Pij_: Product i and Terminal j
*   Vijk: Volume loaded in Vessel i , Product j for Terminal k
*   Bij: Trip for Vessel i for Terminal j
*   Vij_TAT: TAT for Vessel i for Terminal j


In [ ]:
import random
import pandas as pd
from pulp import *

# Set random seed for reproducibility
random.seed(42)


# Parameters
days_in_month = 30

# Terminal 1

P11_capacity = 20000
P21_capacity = 5000

P11_opening_stock_first_day_1 = 7000
P21_opening_stock_first_day_1 = 1000

P11_min_closing_stock = 1000
P21_min_closing_stock = 200

P11_sales = [random.randint(2000, 2500) for _ in range(days_in_month)]       # Generate random sales numbers for each day
P21_sales = [random.randint(800, 1000) for _ in range(days_in_month)]

# Terminal 2

P12_capacity = 18000
P22_capacity = 6000

P12_opening_stock_first_day_1 = 6000
P22_opening_stock_first_day_1 = 4000

P12_min_closing_stock = 1100
P22_min_closing_stock = 300

P12_sales = [random.randint(2200, 3000) for _ in range(days_in_month)]       # Generate random sales numbers for each day
P22_sales = [random.randint(700, 1000) for _ in range(days_in_month)]


#Ship 1


V1_capacity = 12000
V11_turnaround_time = 1                 #lets assume this is the travel time i.e first vessel takes 1 day to reach from supply to terminal 1
V12_turnaround_time = 1
V1_cost = 20

#Ship 2

V2_capacity = 15000
V21_turnaround_time = 2
V22_turnaround_time = 2
V2_cost = 10

In [ ]:


# Create LP problem
problem = LpProblem("Optimize_Delivery", LpMinimize)

# Define decision variables For Vessel 1

V11 = [LpVariable(f"Ship1_terminal1_total_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V111 = [LpVariable(f"Ship1_Prod1_terminal1_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V121 = [LpVariable(f"Ship1_Prod2_terminal1_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]

V12 = [LpVariable(f"Ship1_terminal2_total{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V112 = [LpVariable(f"Ship1_Prod1_terminal2_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V122 = [LpVariable(f"Ship1_Prod2_terminal2_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]

Y_11 = [LpVariable(f"Y_11{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]
B_11 = [LpVariable(f"B_11{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]


Y_12 = [LpVariable(f"Y_12{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]
B_12 = [LpVariable(f"B_12{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]


# Define decision variables For Vessel 2

V21 = [LpVariable(f"Ship2_terminal1_total_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V211 = [LpVariable(f"Ship2_Prod1_terminal1_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V221 = [LpVariable(f"Ship2_Prod2_terminal1_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]

V22 = [LpVariable(f"Ship2_terminal2_total{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V212 = [LpVariable(f"Ship2_Prod1_terminal2_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]
V222 = [LpVariable(f"Ship2_Prod2_terminal2_Vol_{i}", lowBound=0, upBound= V1_capacity/1000, cat='Integer') for i in range(days_in_month)]

Y_21 = [LpVariable(f"Y_21{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]
B_21 = [LpVariable(f"B_21{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]

Y_22 = [LpVariable(f"Y_22{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]
B_22 = [LpVariable(f"B_22{i}", lowBound=0, upBound=1, cat='Integer') for i in range(days_in_month)]

# Define objective function

problem += lpSum(B_11*V1_cost+ B_12*V1_cost + B_21*V2_cost+B_22*V2_cost)



In [ ]:
    #For terminal 1

P11_opening_stock = P11_opening_stock_first_day_1
P21_opening_stock = P21_opening_stock_first_day_1

P11_closing_stock = 0
P12_closing_stock = 0


 #For terminal 2

P12_opening_stock = P12_opening_stock_first_day_1
P22_opening_stock = P22_opening_stock_first_day_1

P12_closing_stock = 0
P22_closing_stock = 0

## now writing the day wise volume equation

for i in range(days_in_month):

    ############################## for terminal 1 ###################################################

     problem += V11[i] == V111[i]+V121[i]   # total volume carried by Ship 1 for terminal 1 on day (i)
     problem += V11[i] <= V1_capacity/1000   # ensure volume carried is less than vessel capacity

     problem += V11[i] +1 <= Y_11[i]+500*B_11[i]        # Y and B is a binary number, 500 is a big number which V11 cannot attain
     problem += V11[i]+ Y_11[i] >= B_11[i]              # this manipulation is done to set B=0, when V=0, B=1 when V>0
     problem += B_11[i]+Y_11[i] == 1


     problem += V21[i] == V211[i]+V221[i]   # total volume carried by Ship 2 for terminal 1 on day (i)
     problem += V21[i] <= V2_capacity/1000   # ensure volume carried is less than vessel capacity

     problem += V21[i] +1 <= Y_21[i]+500*B_21[i]
     problem += V21[i]+ Y_21[i] >= B_21[i]
     problem += B_21[i]+Y_21[i] == 1

    ## volume equation for Product 1 terminal 1

     P11_closing_stock = P11_opening_stock - P11_sales[i] + (1000 * V111[i])+ (1000 * V211[i])
     problem += P11_closing_stock >= P11_min_closing_stock
     problem += P11_closing_stock <= P11_capacity
     P11_opening_stock = P11_closing_stock


    ## volume equation for Product 2 terminal 1

     P21_closing_stock = P21_opening_stock - P21_sales[i] + (1000 * V121[i])+ (1000 * V221[i])
     problem += P21_closing_stock >= P21_min_closing_stock
     problem += P21_closing_stock <= P21_capacity
     P21_opening_stock = P21_closing_stock



     ############################## for terminal 2 ###################################################

     problem += V12[i] == V112[i]+V122[i]   # total volume carried by Ship 1 for terminal 2 on day (i)
     problem += V12[i] <= V1_capacity/1000   # ensure volume carried is less than vessel capacity

     problem += V12[i] +1 <= Y_12[i]+500*B_12[i]
     problem += V12[i]+ Y_12[i] >= B_12[i]
     problem += B_12[i]+Y_12[i] == 1


     problem += V22[i] == V212[i]+V222[i]   # total volume carried by Ship 2 for terminal 2 on day (i)
     problem += V22[i] <= V2_capacity/1000   # ensure volume carried is less than vessel capacity

     problem += V22[i] +1 <= Y_22[i]+500*B_22[i]
     problem += V22[i]+ Y_22[i] >= B_22[i]
     problem += B_22[i]+Y_22[i] == 1

    ## volume equation for Product 1 terminal 2

     P12_closing_stock = P12_opening_stock - P12_sales[i] + (1000 * V112[i])+ (1000 * V212[i])
     problem += P12_closing_stock >= P12_min_closing_stock
     problem += P12_closing_stock <= P12_capacity
     P12_opening_stock = P12_closing_stock


    ## volume equation for Product 2 terminal 2

     P22_closing_stock = P22_opening_stock - P22_sales[i] + (1000 * V122[i])+ (1000 * V222[i])
     problem += P22_closing_stock >= P22_min_closing_stock
     problem += P22_closing_stock <= P22_capacity
     P22_opening_stock = P22_closing_stock


 # Add constraint for turnaround time for :  vessel 1  and terminal 1
     if i < days_in_month - 2*V11_turnaround_time:
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i + 1, i + 2*V11_turnaround_time + 1):
            problem += B_11[i] + B_11[j] <= 1

 # Add constraint for turnaround time for :  vessel 2  and terminal 1
     if i < days_in_month - 2*V21_turnaround_time:
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i + 1, i + V21_turnaround_time + 1):
            problem += B_21[i] + B_21[j] <= 1

 # Add constraint for turnaround time for :  vessel 1  and terminal 2
     if i < days_in_month - 2*V12_turnaround_time:
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i + 1, i + 2*V12_turnaround_time + 1):
            problem += B_12[i] + B_12[j] <= 1

 # Add constraint for turnaround time for :  vessel 2  and terminal 2
     if i < days_in_month - 2*V22_turnaround_time:
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i + 1, i + 2*V22_turnaround_time + 1):
            problem += B_22[i] + B_22[j] <= 1

 # Add constraint for turnaround time for  vessel 1 and terminal 1 and terminal 2 combination
 # Vessel 1 comes to terminal 1 it cannot go to terminal 2 for i+(TAT_temrinal1+TAT_terminal2) days
 #i.e TAT_terminal1 time to go back to supply location to get loaded and TAT_terminal2 to go to teminal 2 after filling.

     if i < days_in_month - int((V11_turnaround_time+V12_turnaround_time)):
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i, i + int((V11_turnaround_time+V12_turnaround_time)) + 1):
            problem += B_11[i] + B_12[j] <= 1

     if i < days_in_month - int((V11_turnaround_time+V12_turnaround_time)):
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i, i + int((V11_turnaround_time+V12_turnaround_time)) + 1):
            problem += B_12[i] + B_11[j] <= 1

    # Add constraint for turnaround time for terminal 1/2 and vessel 2
     if i < days_in_month - int((V21_turnaround_time+ V22_turnaround_time)):
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i, i + int((V21_turnaround_time+ V22_turnaround_time)) + 1):
            problem += B_21[i] + B_22[j] <= 1

     if i < days_in_month - int((V11_turnaround_time+V12_turnaround_time)):
        # If there's an inbound delivery on day i, there can't be another inbound delivery within the TAT days
        for j in range(i, i + int((V11_turnaround_time+V12_turnaround_time)) + 1):
            problem += B_22[i] + B_21[j] <= 1


     problem += B_12[i] + B_22[i] <= 1                #this ensures Vessel 1 and Vessel 2 is not coming to terminal 2 on same day
     problem += B_21[i] + B_11[i] <= 1                #this ensures Vessel 2 and Vessel 1 is not coming to terminal 1 on same day


In [ ]:
# Solve the problem
problem.solve()

1

In [ ]:
B_11 = [int(variable.value()) for variable in B_11]
B_21 = [int(variable.value()) for variable in B_21]
B_12 = [int(variable.value()) for variable in B_12]
B_22 = [int(variable.value()) for variable in B_22]


V111 = [1000 * delivery for delivery in [int(variable.value()) for variable in V111]]
V121 = [1000 * delivery for delivery in [int(variable.value()) for variable in V121]]
V112 = [1000 * delivery for delivery in [int(variable.value()) for variable in V112]]
V122 = [1000 * delivery for delivery in [int(variable.value()) for variable in V122]]


V211 = [1000 * delivery for delivery in [int(variable.value()) for variable in V211]]
V221 = [1000 * delivery for delivery in [int(variable.value()) for variable in V221]]
V212 = [1000 * delivery for delivery in [int(variable.value()) for variable in V212]]
V222 = [1000 * delivery for delivery in [int(variable.value()) for variable in V222]]


In [ ]:
data1 = {
        'Day': list(range(1, days_in_month+1)),
        'B11': B_11,
        'B12': B_12
         }
result_df1 = pd.DataFrame(data1)

    # Print the DataFrame
print(result_df1)

    Day  B11  B12
0     1    0    0
1     2    0    0
2     3    0    0
3     4    0    0
4     5    1    0
5     6    0    0
6     7    0    0
7     8    0    0
8     9    0    0
9    10    0    0
10   11    0    0
11   12    0    0
12   13    0    0
13   14    0    0
14   15    0    0
15   16    0    0
16   17    0    0
17   18    0    0
18   19    0    0
19   20    0    0
20   21    0    0
21   22    0    0
22   23    0    0
23   24    0    0
24   25    0    0
25   26    0    0
26   27    0    0
27   28    0    0
28   29    0    0
29   30    0    0


In [ ]:
# Check if solution is feasible
if problem.status != 1:  # Infeasible or Unbounded
    print("Solution not possible")
else:
    P11_opening_stock = P11_opening_stock_first_day_1
    P12_opening_stock = P12_opening_stock_first_day_1
    P21_opening_stock = P21_opening_stock_first_day_1
    P22_opening_stock = P22_opening_stock_first_day_1

    P11_closing_stocks = []
    P21_closing_stocks = []
    P12_closing_stocks = []
    P22_closing_stocks = []



    cost_vessel1 = []
    cost_vessel2 = []

    for i in range(days_in_month):
        P11_closing_stock = P11_opening_stock - P11_sales[i] + (V111[i]+V211[i])
        P11_closing_stocks.append(P11_closing_stock)

        P21_closing_stock = P21_opening_stock - P21_sales[i] + ( V121[i]+V221[i])
        P21_closing_stocks.append(P21_closing_stock)

        P12_closing_stock = P12_opening_stock - P12_sales[i] + ( V112[i]+V212[i])
        P12_closing_stocks.append(P12_closing_stock)

        P22_closing_stock = P22_opening_stock - P22_sales[i] + ( V122[i]+V222[i])
        P22_closing_stocks.append(P22_closing_stock)

        x = V1_cost * (B_11[i]+B_12[i])  # Cost is incurred only if there's a delivery
        cost_vessel1.append(x)
        y = V2_cost * (B_21[i]+B_22[i])  # Cost is incurred only if there's a delivery
        cost_vessel2.append(y)

        P11_opening_stock = P11_closing_stock
        P21_opening_stock = P21_closing_stock
        P12_opening_stock = P12_closing_stock
        P22_opening_stock = P22_closing_stock


    # Total cost
    total_cost = sum(cost_vessel1)+ sum(cost_vessel2)

    # Create DataFrame
    data = {
        'Day': list(range(1, days_in_month+1)),
        'P11_OS': [P11_opening_stock_first_day_1] + P11_closing_stocks[:-1],
        'P11_Sale': P11_sales,
        'P21_OS': [P21_opening_stock_first_day_1] + P21_closing_stocks[:-1],
        'P21_Sale': P21_sales,
        'V111': V111,
        'V121': V121,
        'V211': V211,
        'V221': V221,
        'P11_CS': P11_closing_stocks,
        'P21_CS': P21_closing_stocks,

        'P12_OS': [P12_opening_stock_first_day_1] + P12_closing_stocks[:-1],
        'P12_Sale': P12_sales,
        'P22_OS': [P22_opening_stock_first_day_1] + P22_closing_stocks[:-1],
        'P22_Sale': P22_sales,
        'V112': V112,
        'V122': V122,
        'V212': V212,
        'V222': V222,
        'P12_CS': P12_closing_stocks,
        'P22_CS': P22_closing_stocks,

        'C1': cost_vessel1,
        'C2': cost_vessel2
    }
    result_df = pd.DataFrame(data)

    # Print the DataFrame
    print(result_df)

    # Print total cost
    print(f"Total Cost: {total_cost}")

result_df.to_csv('out.csv')


    Day  P11_OS  P11_Sale  P21_OS  P21_Sale   V111  V121  V211  V221  P11_CS  \
0     1    7000      2327    1000       939  10000  2000     0     0   14673   
1     2   14673      2057    2061       907      0     0     0     0   12616   
2     3   12616      2012    1154       856      0     0  8000  4000   18604   
3     4   18604      2379    4298       914      0     0     0     0   16225   
4     5   16225      2140    3384       950      0     0     0     0   14085   
5     6   14085      2125    2434       871      0     0     0     0   11960   
6     7   11960      2114    1563       801      0     0     0     0    9846   
7     8    9846      2071     762       994   7000  5000     0     0   14775   
8     9   14775      2377    4768       840      0     0     0     0   12398   
9    10   12398      2052    3928       978      0     0  9000  2000   19346   
10   11   19346      2346    4950       908      0     0     0     0   17000   
11   12   17000      2379    4042       